In [7]:
# Set to your own local working directory
setwd("~/Dropbox (Gladstone)/Documents/nixosjupyterlab")

# Libraries
load.libs <- c(
  "DOSE",
  "GO.db",
  "GSEABase",
  "org.Hs.eg.db", ## Human-specific
  "clusterProfiler",
  "plyr",  ## for ldply
  "dplyr",
  "tidyr",
  "magrittr",
  "stringr",
  "rWikiPathways")
options(install.packages.check.source = "no")
options(install.packages.compile.from.source = "never")
if (!require("pacman")) install.packages("pacman"); library(pacman)
p_load(load.libs, update = FALSE, character.only = TRUE)
status <- sapply(load.libs,require,character.only = TRUE)
if(all(status)){
  print("SUCCESS: You have successfully installed and loaded all required libraries.")
} else{
  cat("ERROR: One or more libraries failed to install correctly. Check the following list for FALSE cases and try again...\n\n")
  status
}

[1] "SUCCESS: You have successfully installed and loaded all required libraries."


In [8]:
# Prepare WikiPathways GMT
wp.hs.gmt <- rWikiPathways::downloadPathwayArchive(organism="Homo sapiens", format = "gmt")
#wp.hs.gmt <-"wikipathways-20191010-gmt-Homo_sapiens.gmt"
wp2gene <- clusterProfiler::read.gmt(wp.hs.gmt)
wp2gene <- wp2gene %>% tidyr::separate(ont, c("name","version","wpid","org"), "%")
wpid2gene <- wp2gene %>% dplyr::select(wpid,gene) #TERM2GENE
wpid2name <- wp2gene %>% dplyr::select(wpid,name) #TERM2NAME
wpid2name<-unique(wpid2name)

# Get GMT from file (e.g., downloaded from Enrichr)
gmt.file <- "OMIM_Disease"
gmt <- clusterProfiler::read.gmt(gmt.file)
gmt.entrez <- bitr(gmt$gene,fromType = "SYMBOL",toType = "ENTREZID",OrgDb = org.Hs.eg.db)
gmt <-gmt %>%
  dplyr::left_join(gmt.entrez, by=c("gene" = "SYMBOL")) %>%
  dplyr::filter(!is.na(ENTREZID)) %>%
  dplyr::select(ont, ENTREZID)
gmt.lists <- gmt %>% group_by(ont) %>%
  dplyr::summarize(cnt = n(),
            genes = list(ENTREZID))
gmt.all.genes <- unique(gmt$ENTREZID)
  
# Apply to each term in GMT  
gmt.wp.overlaps <- ldply(gmt.lists$ont, function(term){

  gmt.term.genes <- gmt %>%
    dplyr::filter(ont == term) %>%
    dplyr::select(ENTREZID)
  
    ## WikiPathways Analysis
  ewp <- clusterProfiler::enricher(
    gene = gmt.term.genes$ENTREZID,
    universe = gmt.all.genes,
    pAdjustMethod = "fdr",
    pvalueCutoff = 0.05, #p.adjust cutoff
    minGSSize = 10,
    maxGSSize = 200,
    TERM2GENE = wpid2gene,
    TERM2NAME = wpid2name)
  #ewp <- DOSE::setReadable(ewp, org.Hs.eg.db, keyType = "ENTREZID")
  #head(ewp, 20)
  
  ## stash results
  if (!is.null(ewp)){
    res <- ewp@result %>%
      dplyr::filter(p.adjust < 0.05)
    
    ovr.genes <- res %>%
      tidyr::separate_rows(geneID, sep="/") %>%
      dplyr::distinct(geneID)
    
    gmt.lists[gmt.lists$ont == term,] %<>%
      dplyr::mutate(wp.cnt = length(res$ID),
             wpids = paste(res$ID, collapse = ","),
             overlap.cnt = length(ovr.genes$geneID),
             overlap.genes = paste(ovr.genes$geneID, collapse = ", ")
      )
  }
})

## Flatten and save
gmt.wp.overlaps <- gmt.wp.overlaps %>%
  rowwise() %>%
  dplyr::mutate(genes = paste(unlist(genes), collapse = ", "))

write.table(gmt.wp.overlaps, "gmt-wp-overlaps.tsv", quote=F, sep="\t", row.names = F)


trying URL 'http://data.wikipathways.org/current/gmt/wikipathways-20191110-gmt-Homo_sapiens.gmt'
Content type 'unknown' length 199785 bytes (195 KB)
downloaded 195 KB

'select()' returned 1:1 mapping between keys and columns
simpleWarning in bitr(gmt$gene, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db): 12.85% of input gene IDs are fail to map...


